# Results Analysis for Diabetic Retinopathy Classification

This notebook provides comprehensive visualization and statistical analysis of training results and cross-dataset evaluation for DR classification models.

**Contents:**
1. Training History Analysis (Loss & Accuracy Curves)
2. Cross-Dataset Evaluation Results
3. Model Comparison (Baseline vs RETFound vs RETFound+LoRA)
4. Generalization Gap Analysis
5. Sample Predictions Visualization
6. Failure Mode Analysis
7. Statistical Significance Testing

**Author:** Muhamad Rihan Rauf Azkiya
**Date:** October 2025

## 1. Setup & Imports

In [ ]:
# Standard library imports
import os
import json
import sys
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from PIL import Image

# Statistical analysis
from scipy import stats
from scipy.stats import wilcoxon, mannwhitneyu
from sklearn.metrics import confusion_matrix, classification_report
from statsmodels.stats.contingency_tables import mcnemar

# Deep learning (for loading models if needed)
import torch

# Configure matplotlib for publication-ready figures
plt.rcParams.update({
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'figure.figsize': (10, 6),
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 16,
    'figure.titleweight': 'bold',
    'axes.grid': True,
    'grid.alpha': 0.3,
    'lines.linewidth': 2,
})

# Set seaborn style
sns.set_style("whitegrid")
sns.set_palette("husl")

# Color palette for consistent model colors
MODEL_COLORS = {
    'baseline': '#1f77b4',      # Blue
    'retfound': '#ff7f0e',      # Orange
    'retfound_lora': '#2ca02c', # Green
}

# DR class names
DR_CLASSES = ['No DR', 'Mild', 'Moderate', 'Severe', 'PDR']

print("✓ All imports successful")
print(f"✓ Working directory: {os.getcwd()}")
print(f"✓ Matplotlib backend: {plt.get_backend()}")

## 2. Configuration Paths

**Modify these paths to match your experiment results:**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# CONFIGURATION - MODIFY THESE PATHS FOR YOUR EXPERIMENTS
# ═══════════════════════════════════════════════════════════════════════

# Project root
PROJECT_ROOT = Path('/Users/rihanrauf/Documents/00. Research/01-diabetic-retinopathy-classification')

# Training history JSON files (output from train_*.py scripts)
TRAINING_HISTORIES = {
    'baseline': PROJECT_ROOT / 'results/baseline/logs/training_history.json',
    'retfound_lora': PROJECT_ROOT / 'results/retfound_lora/logs/training_history.json',
}

# Cross-dataset evaluation JSON files (output from evaluate_cross_dataset.py)
EVALUATION_RESULTS = {
    'baseline': PROJECT_ROOT / 'results/evaluation/baseline/evaluation_results.json',
    'retfound_lora': PROJECT_ROOT / 'results/evaluation/retfound_lora/evaluation_results.json',
}

# Prediction CSV files for sample visualization (optional)
PREDICTION_FILES = {
    'baseline': PROJECT_ROOT / 'results/evaluation/baseline/predictions/APTOS_predictions.csv',
    'retfound_lora': PROJECT_ROOT / 'results/evaluation/retfound_lora/predictions/APTOS_predictions.csv',
}

# Image directory for sample predictions
IMAGE_DIR = PROJECT_ROOT / 'data/aptos/images'

# Output directory for generated plots
OUTPUT_DIR = PROJECT_ROOT / 'results/analysis'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✓ Configuration paths set")
print(f"  Output directory: {OUTPUT_DIR}")

## 3. Load Training History

Load training metrics (loss, accuracy, learning rate) from JSON files.

In [ ]:
def load_training_history(filepath: Path) -> Optional[Dict]:
    """
    Load training history from JSON file.
    
    Expected structure:
    {
        'train_loss': [...],
        'train_acc': [...],
        'val_loss': [...],
        'val_acc': [...],
        'learning_rate': [...],
        'epoch_time': [...],
        'class_accuracies': [[...], ...]
    }
    """
    if not filepath.exists():
        print(f"⚠ Warning: File not found: {filepath}")
        return None
    
    try:
        with open(filepath, 'r') as f:
            history = json.load(f)
        
        # Validate structure
        required_keys = ['train_loss', 'train_acc', 'val_loss', 'val_acc']
        if not all(key in history for key in required_keys):
            print(f"⚠ Warning: Missing required keys in {filepath}")
            return None
        
        print(f"✓ Loaded {filepath.name}: {len(history['train_loss'])} epochs")
        return history
    
    except Exception as e:
        print(f"✗ Error loading {filepath}: {e}")
        return None


# Load all training histories
training_data = {}
for model_name, filepath in TRAINING_HISTORIES.items():
    history = load_training_history(filepath)
    if history is not None:
        training_data[model_name] = history

print(f"\n✓ Loaded {len(training_data)} training histories")
for name in training_data.keys():
    print(f"  - {name}")

## 4. Plot Training & Validation Loss Curves

Compare loss curves across different models to assess convergence.

In [ ]:
def plot_loss_curves(training_data: Dict, save_path: Optional[Path] = None):
    """
    Plot training and validation loss curves for multiple models.
    """
    if not training_data:
        print("⚠ No training data available")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Training Loss
    for model_name, history in training_data.items():
        epochs = range(1, len(history['train_loss']) + 1)
        color = MODEL_COLORS.get(model_name, None)
        ax1.plot(epochs, history['train_loss'], label=model_name.replace('_', ' ').title(),
                color=color, linewidth=2, marker='o', markersize=4, alpha=0.8)
    
    ax1.set_xlabel('Epoch', fontweight='bold')
    ax1.set_ylabel('Training Loss', fontweight='bold')
    ax1.set_title('Training Loss Curves', fontweight='bold', fontsize=14)
    ax1.legend(loc='best', framealpha=0.9)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Validation Loss
    for model_name, history in training_data.items():
        epochs = range(1, len(history['val_loss']) + 1)
        color = MODEL_COLORS.get(model_name, None)
        ax2.plot(epochs, history['val_loss'], label=model_name.replace('_', ' ').title(),
                color=color, linewidth=2, marker='s', markersize=4, alpha=0.8)
    
    ax2.set_xlabel('Epoch', fontweight='bold')
    ax2.set_ylabel('Validation Loss', fontweight='bold')
    ax2.set_title('Validation Loss Curves', fontweight='bold', fontsize=14)
    ax2.legend(loc='best', framealpha=0.9)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()


# Generate plot
plot_loss_curves(training_data, OUTPUT_DIR / 'loss_curves.png')

## 5. Plot Training & Validation Accuracy Curves

Compare accuracy progression and identify best performance epochs.

In [ ]:
def plot_accuracy_curves(training_data: Dict, save_path: Optional[Path] = None):
    """
    Plot training and validation accuracy curves with peak annotations.
    """
    if not training_data:
        print("⚠ No training data available")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Training Accuracy
    for model_name, history in training_data.items():
        epochs = range(1, len(history['train_acc']) + 1)
        color = MODEL_COLORS.get(model_name, None)
        ax1.plot(epochs, history['train_acc'], label=model_name.replace('_', ' ').title(),
                color=color, linewidth=2, marker='o', markersize=4, alpha=0.8)
    
    ax1.set_xlabel('Epoch', fontweight='bold')
    ax1.set_ylabel('Training Accuracy (%)', fontweight='bold')
    ax1.set_title('Training Accuracy Curves', fontweight='bold', fontsize=14)
    ax1.legend(loc='best', framealpha=0.9)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 100])
    
    # Plot 2: Validation Accuracy with peak annotations
    for model_name, history in training_data.items():
        epochs = range(1, len(history['val_acc']) + 1)
        color = MODEL_COLORS.get(model_name, None)
        ax2.plot(epochs, history['val_acc'], label=model_name.replace('_', ' ').title(),
                color=color, linewidth=2, marker='s', markersize=4, alpha=0.8)
        
        # Annotate best accuracy
        best_acc = max(history['val_acc'])
        best_epoch = history['val_acc'].index(best_acc) + 1
        ax2.annotate(f'{best_acc:.1f}%',
                    xy=(best_epoch, best_acc),
                    xytext=(10, 10), textcoords='offset points',
                    ha='left', fontsize=9,
                    bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3),
                    arrowprops=dict(arrowstyle='->', color=color, lw=1.5))
    
    ax2.set_xlabel('Epoch', fontweight='bold')
    ax2.set_ylabel('Validation Accuracy (%)', fontweight='bold')
    ax2.set_title('Validation Accuracy Curves', fontweight='bold', fontsize=14)
    ax2.legend(loc='best', framealpha=0.9)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([0, 100])
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()
    
    # Print summary statistics
    print("\n📊 Best Validation Accuracies:")
    for model_name, history in training_data.items():
        best_acc = max(history['val_acc'])
        best_epoch = history['val_acc'].index(best_acc) + 1
        print(f"  {model_name:20s}: {best_acc:5.2f}% at epoch {best_epoch}")


# Generate plot
plot_accuracy_curves(training_data, OUTPUT_DIR / 'accuracy_curves.png')

## 6. Load Cross-Dataset Evaluation Results

Load evaluation metrics from multiple test datasets.

In [ ]:
def load_evaluation_results(filepath: Path) -> Optional[Dict]:
    """
    Load cross-dataset evaluation results from JSON.
    
    Expected structure:
    {
        'model_info': {...},
        'datasets': {
            'APTOS': {'accuracy': ..., 'precision_macro': ..., ...},
            'Messidor': {...},
        },
        'summary': {
            'mean_accuracy': ...,
            'generalization_gap': ...,
            'best_dataset': ...,
            'worst_dataset': ...
        }
    }
    """
    if not filepath.exists():
        print(f"⚠ Warning: File not found: {filepath}")
        return None
    
    try:
        with open(filepath, 'r') as f:
            results = json.load(f)
        
        # Validate structure
        if 'datasets' not in results:
            print(f"⚠ Warning: Missing 'datasets' key in {filepath}")
            return None
        
        num_datasets = len(results['datasets'])
        print(f"✓ Loaded {filepath.name}: {num_datasets} datasets")
        return results
    
    except Exception as e:
        print(f"✗ Error loading {filepath}: {e}")
        return None


# Load all evaluation results
evaluation_data = {}
for model_name, filepath in EVALUATION_RESULTS.items():
    results = load_evaluation_results(filepath)
    if results is not None:
        evaluation_data[model_name] = results

print(f"\n✓ Loaded {len(evaluation_data)} evaluation results")
for name, results in evaluation_data.items():
    datasets = list(results['datasets'].keys())
    print(f"  - {name}: {datasets}")

## 7. Cross-Dataset Accuracy Comparison

Compare model performance across different test datasets.

In [ ]:
def plot_cross_dataset_comparison(evaluation_data: Dict, save_path: Optional[Path] = None):
    """
    Create grouped bar chart comparing metrics across datasets.
    """
    if not evaluation_data:
        print("⚠ No evaluation data available")
        return
    
    # Extract dataset names (use first model's datasets)
    first_model = list(evaluation_data.values())[0]
    dataset_names = list(first_model['datasets'].keys())
    
    # Metrics to plot
    metrics = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro']
    metric_labels = ['Accuracy', 'F1-Score', 'Precision', 'Recall']
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, (metric, label) in enumerate(zip(metrics, metric_labels)):
        ax = axes[idx]
        
        # Prepare data
        x = np.arange(len(dataset_names))
        width = 0.8 / len(evaluation_data)
        
        # Plot bars for each model
        for i, (model_name, results) in enumerate(evaluation_data.items()):
            values = [results['datasets'][ds][metric] * 100 for ds in dataset_names]
            color = MODEL_COLORS.get(model_name, None)
            offset = (i - len(evaluation_data)/2 + 0.5) * width
            
            bars = ax.bar(x + offset, values, width, label=model_name.replace('_', ' ').title(),
                         color=color, alpha=0.8, edgecolor='black', linewidth=0.5)
            
            # Add value labels on bars
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.1f}', ha='center', va='bottom', fontsize=8)
        
        ax.set_xlabel('Dataset', fontweight='bold')
        ax.set_ylabel(f'{label} (%)', fontweight='bold')
        ax.set_title(f'{label} Across Datasets', fontweight='bold', fontsize=12)
        ax.set_xticks(x)
        ax.set_xticklabels(dataset_names, rotation=15, ha='right')
        ax.legend(loc='best', framealpha=0.9)
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_ylim([0, 100])
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()


# Generate plot
plot_cross_dataset_comparison(evaluation_data, OUTPUT_DIR / 'cross_dataset_comparison.png')

## 8. Generalization Gap Analysis

Visualize the generalization gap (max - min accuracy) for each model.

In [ ]:
def plot_generalization_gap(evaluation_data: Dict, save_path: Optional[Path] = None):
    """
    Plot generalization gap with best/worst dataset annotations.
    """
    if not evaluation_data:
        print("⚠ No evaluation data available")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Generalization Gap Bar Chart
    model_names = []
    gaps = []
    mean_accs = []
    
    for model_name, results in evaluation_data.items():
        model_names.append(model_name.replace('_', ' ').title())
        
        if 'summary' in results:
            gaps.append(results['summary']['generalization_gap'] * 100)
            mean_accs.append(results['summary']['mean_accuracy'] * 100)
        else:
            # Calculate manually if not in summary
            accuracies = [ds['accuracy'] for ds in results['datasets'].values()]
            gaps.append((max(accuracies) - min(accuracies)) * 100)
            mean_accs.append(np.mean(accuracies) * 100)
    
    colors = [MODEL_COLORS.get(m.lower().replace(' ', '_'), None) for m in model_names]
    bars = ax1.bar(model_names, gaps, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Add horizontal line for "acceptable" threshold (e.g., 10%)
    ax1.axhline(y=10, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Target Threshold (10%)')
    
    ax1.set_ylabel('Generalization Gap (%)', fontweight='bold')
    ax1.set_title('Generalization Gap: Max - Min Accuracy', fontweight='bold', fontsize=14)
    ax1.legend(loc='best', framealpha=0.9)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Plot 2: Mean Accuracy with Error Bars (representing gap)
    yerr = [gap/2 for gap in gaps]  # Use half gap as error bar
    ax2.bar(model_names, mean_accs, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax2.errorbar(model_names, mean_accs, yerr=yerr, fmt='none', ecolor='black',
                 capsize=10, capthick=2, linewidth=2, alpha=0.7)
    
    # Add value labels
    for i, (name, mean_acc) in enumerate(zip(model_names, mean_accs)):
        ax2.text(i, mean_acc + yerr[i] + 1,
                f'{mean_acc:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    ax2.set_ylabel('Mean Accuracy (%)', fontweight='bold')
    ax2.set_title('Mean Accuracy Across Datasets (±Gap/2)', fontweight='bold', fontsize=14)
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.set_ylim([0, 100])
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()
    
    # Print summary table
    print("\n📊 Generalization Gap Summary:")
    print("─" * 80)
    print(f"{'Model':<25} {'Mean Acc':<12} {'Gap':<10} {'Best Dataset':<20} {'Worst Dataset':<20}")
    print("─" * 80)
    
    for model_name, results in evaluation_data.items():
        if 'summary' in results:
            mean_acc = results['summary']['mean_accuracy'] * 100
            gap = results['summary']['generalization_gap'] * 100
            best_ds = results['summary']['best_dataset']
            worst_ds = results['summary']['worst_dataset']
        else:
            accuracies = {ds: data['accuracy'] for ds, data in results['datasets'].items()}
            mean_acc = np.mean(list(accuracies.values())) * 100
            gap = (max(accuracies.values()) - min(accuracies.values())) * 100
            best_ds = max(accuracies, key=accuracies.get)
            worst_ds = min(accuracies, key=accuracies.get)
        
        print(f"{model_name:<25} {mean_acc:>6.2f}%     {gap:>6.2f}%   {best_ds:<20} {worst_ds:<20}")
    
    print("─" * 80)


# Generate plot
plot_generalization_gap(evaluation_data, OUTPUT_DIR / 'generalization_gap.png')

## 9. Sample Predictions with Images

Visualize correct and incorrect predictions with actual fundus images.

In [ ]:
def plot_sample_predictions(predictions_csv: Path, image_dir: Path, 
                           num_correct: int = 5, num_incorrect: int = 5,
                           save_path: Optional[Path] = None):
    """
    Plot sample predictions (correct vs incorrect) with images.
    
    Requires:
        - predictions_csv: CSV with columns [image_id, true_label, predicted_label, confidence]
        - image_dir: Directory containing images
    """
    if not predictions_csv.exists():
        print(f"⚠ Predictions file not found: {predictions_csv}")
        return
    
    if not image_dir.exists():
        print(f"⚠ Image directory not found: {image_dir}")
        return
    
    # Load predictions
    df = pd.read_csv(predictions_csv)
    
    # Separate correct and incorrect predictions
    correct_df = df[df['true_label'] == df['predicted_label']].sample(min(num_correct, len(df)))
    incorrect_df = df[df['true_label'] != df['predicted_label']].sample(min(num_incorrect, len(df)))
    
    # Create figure
    fig, axes = plt.subplots(2, max(num_correct, num_incorrect), figsize=(20, 8))
    
    # Plot correct predictions
    for idx, (_, row) in enumerate(correct_df.iterrows()):
        if idx >= num_correct:
            break
        
        # Find image file
        img_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            potential_path = image_dir / f"{row['image_id']}{ext}"
            if potential_path.exists():
                img_path = potential_path
                break
        
        if img_path is None:
            continue
        
        # Load and display image
        img = Image.open(img_path)
        axes[0, idx].imshow(img)
        axes[0, idx].axis('off')
        
        # Add title with prediction info
        true_class = DR_CLASSES[int(row['true_label'])]
        conf = row.get('confidence', 1.0) * 100
        axes[0, idx].set_title(f"✓ {true_class}\nConf: {conf:.1f}%",
                              fontsize=10, color='green', fontweight='bold')
    
    # Plot incorrect predictions
    for idx, (_, row) in enumerate(incorrect_df.iterrows()):
        if idx >= num_incorrect:
            break
        
        # Find image file
        img_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            potential_path = image_dir / f"{row['image_id']}{ext}"
            if potential_path.exists():
                img_path = potential_path
                break
        
        if img_path is None:
            continue
        
        # Load and display image
        img = Image.open(img_path)
        axes[1, idx].imshow(img)
        axes[1, idx].axis('off')
        
        # Add title with prediction info
        true_class = DR_CLASSES[int(row['true_label'])]
        pred_class = DR_CLASSES[int(row['predicted_label'])]
        conf = row.get('confidence', 1.0) * 100
        axes[1, idx].set_title(f"✗ True: {true_class}\nPred: {pred_class} ({conf:.1f}%)",
                              fontsize=10, color='red', fontweight='bold')
    
    # Add row labels
    fig.text(0.02, 0.75, 'CORRECT\nPREDICTIONS', fontsize=14, fontweight='bold',
             color='green', rotation=90, va='center', ha='center')
    fig.text(0.02, 0.25, 'INCORRECT\nPREDICTIONS', fontsize=14, fontweight='bold',
             color='red', rotation=90, va='center', ha='center')
    
    plt.tight_layout(rect=[0.03, 0, 1, 1])
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()


# Generate plot (for first available model)
if PREDICTION_FILES:
    first_pred_file = list(PREDICTION_FILES.values())[0]
    if first_pred_file.exists():
        plot_sample_predictions(first_pred_file, IMAGE_DIR,
                               save_path=OUTPUT_DIR / 'sample_predictions.png')
    else:
        print("⚠ No prediction files found. Run evaluate_cross_dataset.py with --save_predictions")

## 10. Failure Mode Analysis by Class

Analyze which DR classes are most challenging for each model.

In [ ]:
def plot_per_class_performance(evaluation_data: Dict, save_path: Optional[Path] = None):
    """
    Create heatmap showing per-class F1 scores across datasets and models.
    """
    if not evaluation_data:
        print("⚠ No evaluation data available")
        return
    
    # Extract per-class metrics
    # Structure: models x (datasets * classes)
    
    fig, axes = plt.subplots(1, len(evaluation_data), figsize=(8 * len(evaluation_data), 6))
    if len(evaluation_data) == 1:
        axes = [axes]
    
    for idx, (model_name, results) in enumerate(evaluation_data.items()):
        # Build matrix: datasets x classes
        dataset_names = list(results['datasets'].keys())
        matrix = []
        
        for ds_name in dataset_names:
            ds_metrics = results['datasets'][ds_name]
            if 'per_class_metrics' in ds_metrics:
                # Extract F1 scores for each class
                f1_scores = [ds_metrics['per_class_metrics'][str(i)]['f1-score'] * 100 
                           for i in range(5)]
                matrix.append(f1_scores)
            elif 'f1_per_class' in ds_metrics:
                f1_scores = [score * 100 for score in ds_metrics['f1_per_class']]
                matrix.append(f1_scores)
        
        if not matrix:
            print(f"⚠ No per-class metrics found for {model_name}")
            continue
        
        # Create heatmap
        matrix = np.array(matrix)
        ax = axes[idx]
        
        sns.heatmap(matrix, annot=True, fmt='.1f', cmap='RdYlGn', vmin=0, vmax=100,
                   xticklabels=DR_CLASSES, yticklabels=dataset_names,
                   cbar_kws={'label': 'F1-Score (%)'},
                   linewidths=0.5, linecolor='gray', ax=ax)
        
        ax.set_title(f"{model_name.replace('_', ' ').title()}\nPer-Class F1 Scores",
                    fontweight='bold', fontsize=14)
        ax.set_xlabel('DR Class', fontweight='bold')
        ax.set_ylabel('Dataset', fontweight='bold')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {save_path}")
    
    plt.show()
    
    # Identify worst-performing classes
    print("\n📊 Class-wise Performance Analysis:")
    print("─" * 80)
    
    for model_name, results in evaluation_data.items():
        print(f"\n{model_name.upper()}:")
        
        # Aggregate across datasets
        class_f1_scores = {i: [] for i in range(5)}
        
        for ds_metrics in results['datasets'].values():
            if 'per_class_metrics' in ds_metrics:
                for i in range(5):
                    class_f1_scores[i].append(ds_metrics['per_class_metrics'][str(i)]['f1-score'])
            elif 'f1_per_class' in ds_metrics:
                for i, score in enumerate(ds_metrics['f1_per_class']):
                    class_f1_scores[i].append(score)
        
        # Calculate mean F1 per class
        for i, scores in class_f1_scores.items():
            if scores:
                mean_f1 = np.mean(scores) * 100
                std_f1 = np.std(scores) * 100
                print(f"  {DR_CLASSES[i]:<12}: {mean_f1:5.2f}% (±{std_f1:4.2f}%)")


# Generate plot
plot_per_class_performance(evaluation_data, OUTPUT_DIR / 'per_class_performance.png')

## 11. Statistical Significance Testing

Perform rigorous statistical tests to determine if performance differences are significant.

In [ ]:
def statistical_significance_tests(evaluation_data: Dict):
    """
    Perform statistical significance tests between models.
    
    Tests:
    1. McNemar's test (for paired predictions on same test set)
    2. Bootstrap confidence intervals (for accuracy differences)
    3. Wilcoxon signed-rank test (for cross-dataset comparisons)
    """
    if len(evaluation_data) < 2:
        print("⚠ Need at least 2 models for comparison")
        return
    
    print("\n" + "="*80)
    print("STATISTICAL SIGNIFICANCE TESTING")
    print("="*80)
    
    model_names = list(evaluation_data.keys())
    
    # ─────────────────────────────────────────────────────────────────────
    # Test 1: Wilcoxon Signed-Rank Test for Cross-Dataset Performance
    # ─────────────────────────────────────────────────────────────────────
    print("\n1. WILCOXON SIGNED-RANK TEST (Cross-Dataset Accuracy)")
    print("   H0: Models have equal performance across datasets")
    print("   H1: Models have different performance")
    print("─" * 80)
    
    for i in range(len(model_names)):
        for j in range(i+1, len(model_names)):
            model1, model2 = model_names[i], model_names[j]
            results1 = evaluation_data[model1]
            results2 = evaluation_data[model2]
            
            # Get accuracies across common datasets
            common_datasets = set(results1['datasets'].keys()) & set(results2['datasets'].keys())
            
            if len(common_datasets) < 2:
                print(f"  ⚠ Not enough common datasets for {model1} vs {model2}")
                continue
            
            accs1 = [results1['datasets'][ds]['accuracy'] for ds in common_datasets]
            accs2 = [results2['datasets'][ds]['accuracy'] for ds in common_datasets]
            
            # Perform Wilcoxon test
            try:
                statistic, p_value = wilcoxon(accs1, accs2)
                
                print(f"\n  {model1} vs {model2}:")
                print(f"    Datasets: {list(common_datasets)}")
                print(f"    Mean Acc: {np.mean(accs1)*100:.2f}% vs {np.mean(accs2)*100:.2f}%")
                print(f"    Wilcoxon Statistic: {statistic:.4f}")
                print(f"    P-value: {p_value:.4f}")
                
                if p_value < 0.001:
                    print(f"    Result: *** HIGHLY SIGNIFICANT (p < 0.001) ***")
                elif p_value < 0.01:
                    print(f"    Result: ** SIGNIFICANT (p < 0.01) **")
                elif p_value < 0.05:
                    print(f"    Result: * SIGNIFICANT (p < 0.05) *")
                else:
                    print(f"    Result: NOT SIGNIFICANT (p >= 0.05)")
            
            except Exception as e:
                print(f"  ✗ Test failed for {model1} vs {model2}: {e}")
    
    # ─────────────────────────────────────────────────────────────────────
    # Test 2: Bootstrap Confidence Intervals for Accuracy Difference
    # ─────────────────────────────────────────────────────────────────────
    print("\n\n2. BOOTSTRAP CONFIDENCE INTERVALS (95% CI for Accuracy Difference)")
    print("   Number of bootstrap samples: 10,000")
    print("─" * 80)
    
    n_bootstrap = 10000
    
    for i in range(len(model_names)):
        for j in range(i+1, len(model_names)):
            model1, model2 = model_names[i], model_names[j]
            results1 = evaluation_data[model1]
            results2 = evaluation_data[model2]
            
            # Get predictions for common datasets
            common_datasets = set(results1['datasets'].keys()) & set(results2['datasets'].keys())
            
            if not common_datasets:
                continue
            
            # Aggregate predictions across datasets
            all_preds1, all_preds2, all_true = [], [], []
            
            for ds in common_datasets:
                ds1 = results1['datasets'][ds]
                ds2 = results2['datasets'][ds]
                
                if 'predictions' in ds1 and 'ground_truth' in ds1:
                    all_preds1.extend(ds1['predictions'])
                    all_preds2.extend(ds2['predictions'])
                    all_true.extend(ds1['ground_truth'])
            
            if not all_true:
                print(f"  ⚠ No predictions available for {model1} vs {model2}")
                continue
            
            all_preds1 = np.array(all_preds1)
            all_preds2 = np.array(all_preds2)
            all_true = np.array(all_true)
            
            # Bootstrap
            differences = []
            n_samples = len(all_true)
            
            for _ in range(n_bootstrap):
                # Sample with replacement
                indices = np.random.choice(n_samples, size=n_samples, replace=True)
                
                acc1 = np.mean(all_preds1[indices] == all_true[indices])
                acc2 = np.mean(all_preds2[indices] == all_true[indices])
                
                differences.append(acc1 - acc2)
            
            differences = np.array(differences)
            
            # Calculate 95% CI
            ci_lower = np.percentile(differences, 2.5) * 100
            ci_upper = np.percentile(differences, 97.5) * 100
            mean_diff = np.mean(differences) * 100
            
            print(f"\n  {model1} vs {model2}:")
            print(f"    Samples: {n_samples}")
            print(f"    Mean Accuracy Difference: {mean_diff:+.2f}%")
            print(f"    95% CI: [{ci_lower:+.2f}%, {ci_upper:+.2f}%]")
            
            if ci_lower > 0:
                print(f"    Result: {model1} SIGNIFICANTLY BETTER (CI does not include 0)")
            elif ci_upper < 0:
                print(f"    Result: {model2} SIGNIFICANTLY BETTER (CI does not include 0)")
            else:
                print(f"    Result: NO SIGNIFICANT DIFFERENCE (CI includes 0)")
    
    # ─────────────────────────────────────────────────────────────────────
    # Test 3: McNemar's Test (for paired predictions)
    # ─────────────────────────────────────────────────────────────────────
    print("\n\n3. MCNEMAR'S TEST (Paired Predictions)")
    print("   H0: Models have equal error rates")
    print("   H1: Models have different error rates")
    print("─" * 80)
    
    for i in range(len(model_names)):
        for j in range(i+1, len(model_names)):
            model1, model2 = model_names[i], model_names[j]
            results1 = evaluation_data[model1]
            results2 = evaluation_data[model2]
            
            # Get predictions for common datasets
            common_datasets = set(results1['datasets'].keys()) & set(results2['datasets'].keys())
            
            # Aggregate predictions
            all_preds1, all_preds2, all_true = [], [], []
            
            for ds in common_datasets:
                ds1 = results1['datasets'][ds]
                ds2 = results2['datasets'][ds]
                
                if 'predictions' in ds1 and 'ground_truth' in ds1:
                    all_preds1.extend(ds1['predictions'])
                    all_preds2.extend(ds2['predictions'])
                    all_true.extend(ds1['ground_truth'])
            
            if not all_true:
                continue
            
            all_preds1 = np.array(all_preds1)
            all_preds2 = np.array(all_preds2)
            all_true = np.array(all_true)
            
            # Create contingency table
            # Rows: Model1 correct/incorrect, Cols: Model2 correct/incorrect
            correct1 = (all_preds1 == all_true)
            correct2 = (all_preds2 == all_true)
            
            both_correct = np.sum(correct1 & correct2)
            both_incorrect = np.sum(~correct1 & ~correct2)
            m1_correct_m2_incorrect = np.sum(correct1 & ~correct2)
            m1_incorrect_m2_correct = np.sum(~correct1 & correct2)
            
            # McNemar's test focuses on discordant pairs
            table = [[both_correct, m1_correct_m2_incorrect],
                    [m1_incorrect_m2_correct, both_incorrect]]
            
            try:
                result = mcnemar(table, exact=False, correction=True)
                
                print(f"\n  {model1} vs {model2}:")
                print(f"    Both correct: {both_correct}")
                print(f"    Both incorrect: {both_incorrect}")
                print(f"    {model1} correct, {model2} incorrect: {m1_correct_m2_incorrect}")
                print(f"    {model1} incorrect, {model2} correct: {m1_incorrect_m2_correct}")
                print(f"    McNemar Statistic: {result.statistic:.4f}")
                print(f"    P-value: {result.pvalue:.4f}")
                
                if result.pvalue < 0.001:
                    print(f"    Result: *** HIGHLY SIGNIFICANT (p < 0.001) ***")
                elif result.pvalue < 0.01:
                    print(f"    Result: ** SIGNIFICANT (p < 0.01) **")
                elif result.pvalue < 0.05:
                    print(f"    Result: * SIGNIFICANT (p < 0.05) *")
                else:
                    print(f"    Result: NOT SIGNIFICANT (p >= 0.05)")
                
                # Interpret which model is better
                if result.pvalue < 0.05:
                    if m1_correct_m2_incorrect > m1_incorrect_m2_correct:
                        print(f"    Interpretation: {model1} performs significantly BETTER")
                    else:
                        print(f"    Interpretation: {model2} performs significantly BETTER")
            
            except Exception as e:
                print(f"  ✗ Test failed: {e}")
    
    print("\n" + "="*80)
    print("LEGEND:")
    print("  ***: p < 0.001 (highly significant)")
    print("  **:  p < 0.01  (significant)")
    print("  *:   p < 0.05  (significant)")
    print("       p >= 0.05 (not significant)")
    print("="*80 + "\n")


# Perform statistical tests
statistical_significance_tests(evaluation_data)

## Summary

This notebook provides comprehensive analysis of:
- **Training dynamics**: Loss/accuracy curves show convergence behavior
- **Cross-dataset performance**: Comparison across multiple test sets
- **Generalization ability**: Gap between best and worst performance
- **Qualitative analysis**: Sample predictions with images
- **Failure modes**: Per-class performance breakdown
- **Statistical rigor**: Significance testing with multiple methods

All visualizations are publication-ready (300 DPI, proper styling).